In [1]:
from scipy.integrate import odeint
from pylab import *
from lmfit import *
from scipy import stats
%matplotlib inline

In [2]:
# Calculate all cases

# Normalization
norm = {'PCBnone':0,'PCBall':1,'PCB118':2,'PCB153':3,'PCB138':4}

site = {"3C":0,"6C":1}
    
for k, v in sorted(norm.items()):
    print k,v

site = 'Site 3C'

PCB118 2
PCB138 4
PCB153 3
PCBall 1
PCBnone 0


In [3]:
# Load data for the selected site
if site == 'Site 3C':
    A=np.array([\
    [1981.27,1.34,0.07,0.004,0.0081,0.0062,0.0076],\
    [1992.51,0.71,0.25,0.012,0.0066,0.0049,0.0064],\
    [2003.58,0.39,0.32,0.036,0.0050,0.0037,0.0050],\
    [2009.23,0.38,0.52,0.069,0.0076,0.0076,0.0085],\
    [2010.31,0.41,0.52,0.085,0.0075,0.0072,0.0079]])
if site == 'Site 6C':
    A=np.array([\
    [1992.52,3.07,0.67,0.026,0.0182,0.0119,0.0152],\
    [2003.58,2.75,0.87,0.072,0.0177,0.0122,0.0158],\
    [2009.23,2.37,0.75,0.095,0.0172,0.0095,0.0134],\
    [2010.31,1.16,0.43,0.046,0.0079,0.0054,0.0065]])

t = A[:,0]
DDE=A[:,1]
DDMU=A[:,2]
DDNU=A[:,3]
PCB118=A[:,4]
PCB153=A[:,5]
PCB138=A[:,6]

print("Data from {0} with shape {1}".format(site,A.shape))
print('\n'.join([''.join(['{:5.4f}  '.format(item) for item in row]) 
      for row in A]))

# guess at std. error in measurements per conv. with R.E.
stde_DDE  = 0.05          #RE suggests 0.05
stde_DDMU = 0.1          #RE suggested 0.1
stde_DDNU = 0.1          #RE suggested 0.1
stde_PCB  = 0.2          # std. error of measurements is around 0.2
eps_DDE  = stde_DDE*DDE  
eps_DDMU = stde_DDMU*DDMU #RE suggested 0.1
eps_DDNU = stde_DDNU*DDNU #RE suggested 0.2

Data from Site 3C with shape (5, 7)
1981.2700  1.3400  0.0700  0.0040  0.0081  0.0062  0.0076  
1992.5100  0.7100  0.2500  0.0120  0.0066  0.0049  0.0064  
2003.5800  0.3900  0.3200  0.0360  0.0050  0.0037  0.0050  
2009.2300  0.3800  0.5200  0.0690  0.0076  0.0076  0.0085  
2010.3100  0.4100  0.5200  0.0850  0.0075  0.0072  0.0079  


In [4]:
# Objective function to minimize, fit through origin, estimate decay rate
def residual(params, t, data, eps_data):
    C0 = data[0]
    k = params['decay_rate']
    model = C0*np.exp(-k*(t-t[0]))
    return (data-model)**2/eps_data**2

# Objective function to minimize, estimate origin and decay rate
def residual2(params, t, data, eps_data):
    C0 = params['init_conc']
    k = params['decay_rate']
    model = C0*np.exp(-k*(t-t[0]))
    return (data-model)**2/eps_data**2

In [11]:
# Non-linear fit through initial conc
print('\nNon-linear fit, forced through C0')
data = DDE
eps_data = eps_DDE
params = Parameters()

params.add('decay_rate',value = 0.4, min =0., max = 1.)
out = minimize(residual, params, method='leastsq', args=(t,data,eps_data))

# Get the fitted rates
k1f=out.params['decay_rate'].value
k1e = out.params['decay_rate'].stderr

# Get fit metrics
chis = out.chisqr
chir = out.redchi
report_fit(out)
# Calculate fit profile
Chat = data[0]*np.exp(-k1f*t)
r2 = np.corrcoef(data[1:], Chat[1:])[0,1]**2
print("r2 = ",r2)

# Non-linear fit, fit initial conc
print('\nNon-linear fit, and fit initial conc')
data = DDE
eps_data = eps_DDE
params = Parameters()
params.add('decay_rate',value = 0.4, min =0., max = 1.)
params.add('init_conc',value = data[0], min =0., max = 5.*data[0])

out = minimize(residual2, params, method='leastsq', args=(t,data,eps_data))

# Get the fitted rates
k1f=out.params['decay_rate'].value
k1e = out.params['decay_rate'].stderr
Cof=out.params['init_conc'].value
Coe = out.params['init_conc'].stderr

# Get fit metrics
chis = out.chisqr
chir = out.redchi
report_fit(out)

# Calculate fit profile
Chat = Cof*np.exp(-k1f*t)
r2 = np.corrcoef(data, Chat)[0,1]**2
print("r2 = ",r2)

slope, intercept, r_value, p_value, std_err = stats.linregress(t,log(data))
C0hat = exp(intercept+slope*t[0])
print('\nLinear regression: slope, C0hat, r**2, p, std_err:')
print(slope, C0hat, r_value**2, p_value, std_err)


Non-linear fit, forced through C0
[[Fit Statistics]]
    # function evals   = 32
    # data points      = 5
    # variables        = 1
    chi-square         = 385.561
    reduced chi-square = 96.390
    Akaike info crit   = 23.726
    Bayesian info crit = 23.336
[[Variables]]
    decay_rate:   0.04791219 +/- 0.001783 (3.72%) (init= 0.4)
[[Correlations]] (unreported correlations are <  0.100)
('r2 = ', 0.90885351855981733)

Non-linear fit, and fit initial conc
[[Fit Statistics]]
    # function evals   = 72
    # data points      = 5
    # variables        = 2
    chi-square         = 283.454
    reduced chi-square = 94.485
    Akaike info crit   = 24.188
    Bayesian info crit = 23.407
[[Variables]]
    decay_rate:   0.04302361 +/- 0.004949 (11.50%) (init= 0.4)
    init_conc:    1.18868534 +/- 0.135436 (11.39%) (init= 1.34)
[[Correlations]] (unreported correlations are <  0.100)
    C(decay_rate, init_conc)     =  0.918 
('r2 = ', 0.97245685995210318)

Linear regression: slope, C0hat,